In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException, NoSuchFrameException
from selenium.webdriver.common.keys import Keys
from selenium_stealth import stealth
import undetected_chromedriver as uc 
import subprocess
import re
import os
import glob
import pandas as pd
import time
from time import sleep
import random
import pdb

In [ ]:
def create_driver(agent="user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"):
    try:
        print('driver started')
        chrome_options = Options()
        chrome_options.add_argument(agent)
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        driver.maximize_window() #max size for consistency with element names
        stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="MacIntel",
            webgl_vendor="Apple Inc.",
            renderer="Apple GPU",
            fix_hairline=True,
        )
        return driver
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
def process_accounts():
    # Load the accounts data
    accounts_df = pd.read_csv('../reference/accounts.csv')

    # Loop through the accounts
    for index, account in accounts_df.iterrows():
        # Check if account status is NaN
        if pd.isnull(account['email_acc_status']):
            # Start a new session
            driver = create_driver(account['user_agent'])

            # Navigate to ProtonMail
            driver.get('https://account.proton.me/mail/signup')
            sleep(3+3*random.random())
            # Switch to iframe 1 to interact with the 'email' field
            driver.switch_to.frame(1)
            username_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'email')))
            # Extract the part of the username before the '@'
            username_prefix = account['username'].split('@')[0]
            # Send the username one character at a time with a random delay between each
            for character in username_prefix:
                username_field.send_keys(character)
                sleep(random.random())
            # Switch back to the default content to interact with the 'password' and 'repeat-password' fields
            driver.switch_to.default_content()

            password_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'password')))
            password_field.send_keys(account['password'])
            sleep(1+2*random.random())

            repeat_password_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'repeat-password')))
            repeat_password_field.send_keys(account['password'])
            sleep(1+2*random.random())
            # Submit the form
            account_create = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[type="submit"]')))
            account_create.click()

            # Click the 'Continue with Free' button
            continue_free_button = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.NAME, 'Continue with Free')))
            continue_free_button.click()
            sleep(5+5*random.random())

            # Click the 'Next' button
            next_button = WebDriverWait(driver, 160).until(EC.element_to_be_clickable((By.NAME, 'Next')))
            next_button.click()
            sleep(5+5*random.random())

            # Click the 'Maybe later' button
            maybe_later_button = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.NAME, 'Maybe later')))
            maybe_later_button.click()
            sleep(5+5*random.random())

            # Press escape instead of clicking the specified button
            ActionChains(driver).send_keys(Keys.ESCAPE).perform()
            sleep(5+5*random.random())


            # Click the 'Next' button
            next_button = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.NAME, 'Next')))
            next_button.click()
            sleep(2+2*random.random())

            # Click the 'Next' button
            next_button = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.NAME, 'Next')))
            next_button.click()
            sleep(2+2*random.random())

            # Click the 'Next' button
            next_button = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.NAME, 'Get started')))
            next_button.click()
            sleep(2+2*random.random())

            # Press escape instead of clicking the specified button
            ActionChains(driver).send_keys(Keys.ESCAPE).perform()

            # Update the 'acc_status' column in the DataFrame
            accounts_df.loc[index, 'email_acc_status'] = 1

            # Write the DataFrame back to the CSV file
            accounts_df.to_csv('../reference/accounts.csv', index=False)

process_accounts()

In [ ]:
def get_verification_code_from_file(download_dir='/Users/adamhunter/Downloads'):
    # Get a list of all the pgp*.txt files in the download directory
    files = glob.glob(os.path.join(download_dir, 'pgp*.txt'))

    # Find the most recent file
    latest_file = max(files, key=os.path.getctime)

    # Open the file and read the contents
    with open(latest_file, 'r') as f:
        contents = f.read()

    # Use regex to find the verification code in the line that starts with "Subject:"
    match = re.search(r'Subject:.*?(\d{6})', contents)
    if match:
        verification_code = match.group(1)
    else:
        print("No verification code found in email.")
        verification_code = None

    return verification_code

def grab_verification(driver, username, password):
    # Save the handle of the original tab
    original_tab = driver.current_window_handle

    # Open a new tab
    driver.execute_script("window.open('');")

    # Switch to the new tab (it's always the last one)
    driver.switch_to.window(driver.window_handles[-1])

    # Navigate to ProtonMail
    driver.get('https://mail.protonmail.com/login')
    # Input username
    username_field = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, 'username')))
    sleep(random.random()*2)    
    username_field.send_keys(username)

    password_field = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, 'password')))
    sleep(random.random()*3)    
    password_field.send_keys(password)

    # Submit form
    login_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.button-large')))
    sleep(random.random()*3)    
    login_button.click()
    print('clicked login to email')

    # Click the first email in the inbox
    first_email = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.item-container-wrapper:nth-child(1) .item-subject .max-w100')))
    first_email.click()

    # Click 'More options' button
    more_options_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.button:nth-child(11)')))
    more_options_button.click()

    # Click 'View headers' button
    view_headers_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.dropdown-item:nth-child(10) .flex-item-fluid')))
    view_headers_button.click()

    # Click 'Download' button
    download_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.button-solid-norm:nth-child(2)')))
    download_button.click()

    

    # Switch back to the original tab
    driver.switch_to.window(original_tab)

    sleep(3)

    verification_code = get_verification_code_from_file()

    # Input verification code
    verification_field = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, 'input__email_verification_pin')))
    sleep(random.random()*3)    
    verification_field.send_keys(verification_code)
    sleep(random.random())

    # Click the verify button
    verify_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, 'email-pin-submit-button')))

    # Create an ActionChains object mouse movement to avoid detection
    actions = ActionChains(driver)

    # Move to the sign in button and click it
    actions.move_to_element(verify_button).click().perform()

def create_linkedin_account(driver, username, password, firstname, lastname):
    # Navigate to LinkedIn's registration page
    driver.get('https://www.linkedin.com/start/join')
    sleep(10+10*random.random())
    # Fill in the registration form
    email_input = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.NAME, 'email-address')))
    email_input.send_keys(username)
    password_input = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.NAME, 'password')))
    password_input.send_keys(password)
    sleep(5+5*random.random())

    # Submit the form
    submit_button = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH, '//button[@type="submit"]')))
    submit_button.click()

    firstname_input = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.ID, 'first-name')))
    firstname_input.send_keys(firstname)
    sleep(5+5*random.random())

    lastname_input = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.ID, 'last-name')))
    lastname_input.send_keys(lastname)

    # Click the 'Join' button
    join_button = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.ID, 'join-form-submit')))
    join_button.click()
    sleep(5+5*random.random())

    # Click the 'Next' button
    next_button = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.NAME, 'Next')))
    next_button.click()
    sleep(2+2*random.random())

    # Enter 'data analyst' into the specified input
    title_input = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.ID, 'typeahead-input-for-title')))
    title_input.send_keys('Data Analyst')
    ActionChains(driver).send_keys(Keys.ENTER).perform()

    sleep(2+2*random.random())

    # Click the employment type input and simulate keyboard actions
    employment_type_input = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.ID, 'typeahead-input-for-employment-type-picker')))
    employment_type_input.click()
    ActionChains(driver).send_keys(Keys.DOWN).perform()
    ActionChains(driver).send_keys(Keys.ENTER).perform()

    # Generate a random 5 letter string
    random_string = ''.join(random.choices(string.ascii_lowercase, k=5))

    # Enter the random string into the company input
    company_input = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.ID, 'typeahead-input-for-company')))
    company_input.send_keys(random_string)

    if "Let's do a quick verification" in driver.page_source:
        grab_verification(driver, username, password)
    # Get the verification code from the most recent pgp*.txt file
    elif "Let's do a quick security check" in driver.page_source: #captcha page
        driver.switch_to.window(driver.current_window_handle) #brings page to focus for you
        sleep(30)

    sleep(2+2*random.random())

    # Wait for a few seconds to allow the page to load
    sleep(5+5*random.random())

    # Return the driver object for further operations
    return driver

def process_linkedin_accounts():
    # Load the accounts data
    accounts_df = pd.read_csv('../reference/accounts.csv')

    # Loop through the accounts
    for index, account in accounts_df.iterrows():
        # Check if account status is NaN
        if pd.isnull(account['acc_status']) and account['email_acc_status'] == 1:
            # Start a new session
            driver = create_driver(account['user_agent'])
            # Create LinkedIn account
            driver = create_linkedin_account(driver, account['username'], account['password'], account['firstname'], account['lastname'])

process_linkedin_accounts()
